In [1]:
!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6

Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6
     |████████████████████████████████| 1.5MB 3.3MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 58 kB 2.0 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-py3-none-any.whl size=72859 sha256=ba3f1dc967204eacf2eef7325a5b5ec8dbb6bb3c46f15d2386b94462d49f962b
  Stored in directory: /root/.cache/pip/wheels/01/3e/ff/77407ebac3ef71a79b9166a8382aecf88415a0bcbe3c095a01
Successfully built kaggle


In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle -v


Kaggle API 1.5.6


In [3]:
!kaggle competitions download -c classification-animal-classification


  0% 0.00/999 [00:00<?, ?B/s]
100% 999/999 [00:00<00:00, 802kB/s]


In [4]:
!unzip classification-animal-classification.zip

Archive:  classification-animal-classification.zip
  inflating: test_des.csv            
  inflating: test_summit_form.csv    
  inflating: train.csv               


In [78]:
import torch
import torch.optim as optim 
import torch.nn.functional as F 
# 소프트맥스 함수를 사용하기 위함. 
import numpy as np 
# numpy 
import pandas as pd
# pandas 
from sklearn.svm import SVC

torch.manual_seed(1)

data_zoo = pd.read_csv('train.csv')
data_zoo.head()
#print(len(data_zoo))

x_data = data_zoo.iloc[:,:-1]
#x_data.head()

y_data = data_zoo.iloc[:,[-1]]
#y_data.head()

print(x_data.shape)
print(y_data.shape)

x_train = torch.FloatTensor(x_data.values)
y_train = torch.LongTensor(y_data.values)

print(x_train.shape)
y_train = y_train.view(80)
print(y_train.shape)

(80, 16)
(80, 1)
torch.Size([80, 16])
torch.Size([80])


In [79]:
classifier = SVC(kernel = "rbf", C = 1.0, gamma = 0.1)
classifier.fit(x_train, y_train)
print("svc training accuracy :", classifier.score(x_train, y_train))


nb_class = 7
nb_data = len(y_train)

w = torch.zeros((16, nb_class),requires_grad=True)
b = torch.zeros(1,requires_grad=True)

optimizer = optim.SGD([w,b], lr = 0.05,momentum=0.9)


nb_epochs = 200000

for epoch in range(nb_epochs+1):

    hypothesis = F.softmax(x_train.matmul(w)+b, dim=1) # 소프트맥스 함수 / 가설함수 

    y_one_hot = torch.zeros(nb_data, nb_class)
    y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)
    cost = (y_one_hot * -torch.log(F.softmax(hypothesis, dim=1))).sum(dim=1).mean()
    
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    if epoch % 10000 == 0:
        print('epoch {:4d}/{} cost : {:.6f}'.format(
            epoch, nb_epochs,cost.item()
        ))


training accuracy : 1.0
epoch    0/200000 cost : 1.945910
epoch 10000/200000 cost : 1.273011
epoch 20000/200000 cost : 1.272407
epoch 30000/200000 cost : 1.166502
epoch 40000/200000 cost : 1.165967
epoch 50000/200000 cost : 1.165793
epoch 60000/200000 cost : 1.165704
epoch 70000/200000 cost : 1.165650
epoch 80000/200000 cost : 1.165614
epoch 90000/200000 cost : 1.165588
epoch 100000/200000 cost : 1.165568
epoch 110000/200000 cost : 1.165553
epoch 120000/200000 cost : 1.165540
epoch 130000/200000 cost : 1.165530
epoch 140000/200000 cost : 1.165521
epoch 150000/200000 cost : 1.165514
epoch 160000/200000 cost : 1.165508
epoch 170000/200000 cost : 1.165502
epoch 180000/200000 cost : 1.165497
epoch 190000/200000 cost : 1.165493
epoch 200000/200000 cost : 1.165489


In [80]:
hypothesis = F.softmax(x_train.matmul(w) + b, dim=1) # or .mm or @
predict = torch.argmax(hypothesis, dim=1)
#print(hypothesis)
#print(predict)
#print(y_train)

# 정확도 계산 
correct_prediction = predict.float() == y_train
#print(correct_prediction)
accuracy = correct_prediction.sum().item() / len(correct_prediction)
print('The model has an accuracy of {:2.2f}% for the training set.'.format(accuracy * 100))


The model has an accuracy of 100.00% for the training set.


In [82]:
test = pd.read_csv('test_des.csv',header=None)
x_data = test.loc[:,0:15]
x_data = np.array(x_data)
x_test = torch.FloatTensor(x_data)
hypothesis = F.softmax(x_test.matmul(w)+b,dim=1)
predict = torch.argmax(hypothesis,dim=1)
print(predict)

y_pred = classifier.predict(x_test)
#print(y_pred)
y_test = torch.FloatTensor(y_pred)
y_test = y_test.int()
print(y_test)


tensor([1, 3, 1, 0, 6, 3, 1, 5, 4, 1, 4, 3, 0, 0, 1, 0, 5, 0, 1, 1],
       grad_fn=<NotImplemented>)
tensor([6, 3, 1, 0, 6, 3, 1, 5, 4, 0, 4, 3, 0, 0, 1, 0, 5, 0, 6, 1],
       dtype=torch.int32)


In [0]:
predict = y_test.detach().numpy().reshape(-1,1)
id = np.array([i for i in range(20)]).reshape(-1,1)
result = np.hstack([id,predict])
df = pd.DataFrame(result,columns=["Id","Category"])
df.to_csv("result.csv",index=False,header=True)


In [84]:
predict

array([[6],
       [3],
       [1],
       [0],
       [6],
       [3],
       [1],
       [5],
       [4],
       [0],
       [4],
       [3],
       [0],
       [0],
       [1],
       [0],
       [5],
       [0],
       [6],
       [1]], dtype=int32)

In [85]:
result

array([[ 0,  6],
       [ 1,  3],
       [ 2,  1],
       [ 3,  0],
       [ 4,  6],
       [ 5,  3],
       [ 6,  1],
       [ 7,  5],
       [ 8,  4],
       [ 9,  0],
       [10,  4],
       [11,  3],
       [12,  0],
       [13,  0],
       [14,  1],
       [15,  0],
       [16,  5],
       [17,  0],
       [18,  6],
       [19,  1]])

In [86]:
!kaggle competitions submit -c classification-animal-classification -f result.csv -m "14010974_이기택"

100% 102/102 [00:01<00:00, 58.1B/s]
Successfully submitted to Classification : Animal Classification